<a href="https://colab.research.google.com/github/macgyver121/DADS7202_final_project/blob/main/prototypeV3_evaluate%2Bmd_k2t_24_12_65.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check GPU

In [1]:
# List all NVIDIA GPUs as avaialble in this computer (or Colab's session)
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-76866b93-494d-010c-fce5-51c170827233)


In [2]:
import sys
print( f"Python {sys.version}\n" )

import numpy as np
print( f"NumPy {np.__version__}\n" )

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print( f"TensorFlow {tf.__version__}" )
print( f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}" )

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print( f"TensorFlow detected { len(gpus) } GPU(s):" )
for i, gpu in enumerate(gpus):
  print( f".... GPU No. {i}: Name = {gpu.name} , Type = {gpu.device_type}" )

Python 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]

NumPy 1.21.6

TensorFlow 2.9.2
tf.keras.backend.image_data_format() = channels_last
TensorFlow detected 1 GPU(s):
.... GPU No. 0: Name = /physical_device:GPU:0 , Type = GPU


Install pytesseract

In [3]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install -q transformers datasets
!pip install -q pytorch-lightning wandb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,242 kB/s)
debconf: unable to initi

     |████████████████████████████████| 5.8 MB 34.5 MB/s 
     |████████████████████████████████| 452 kB 74.7 MB/s 
     |████████████████████████████████| 7.6 MB 70.1 MB/s 
     |████████████████████████████████| 182 kB 77.2 MB/s 
     |████████████████████████████████| 212 kB 82.0 MB/s 
     |████████████████████████████████| 132 kB 80.9 MB/s 
     |████████████████████████████████| 127 kB 75.5 MB/s 
     |████████████████████████████████| 800 kB 29.5 MB/s 
     |████████████████████████████████| 1.9 MB 61.8 MB/s 
     |████████████████████████████████| 512 kB 75.1 MB/s 
     |████████████████████████████████| 125 kB 78.1 MB/s 
     |████████████████████████████████| 174 kB 76.4 MB/s 
     |████████████████████████████████| 182 kB 80.9 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 173 kB 78.9 MB/s 
     |████████████████████████████████| 168 kB 81.0 MB/s 
     |████████████████████████████████| 168 kB 84.8 MB/s 
     |██████████

Reset runtime

In [ ]:
import os
os.kill(os.getpid(), 9)

Import library

In [1]:
import cv2
import pytesseract
import shutil
import os
import random
from PIL import Image

import re
import csv
import os

import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

Mount drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import re
import csv
import os
import pyarrow as pa
import torch
import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

## Create Data from Chart2text data

In [174]:
List_key = []
# Open the CSV file for reading
for i in range(1000): 
  with open('/content/drive/MyDrive/DADS7202_final/Chart2text/data/'+str(i)+'.csv', 'r') as d:
 # Read the contents of the file into a list of rows
    rows = list(csv.reader(d))
# Concatenate the rows into a single string
    csv_string = '\n'.join(['\t'.join(row) for row in rows])
    csv_string = re.sub('\s+',' ',csv_string)
    List_key.append(csv_string)
len(List_key)

1000

In [175]:
import pickle

with open('List_key_0to1000.pkl', 'wb') as f:
    pickle.dump(List_key, f)

Load

In [176]:
import pickle

with open('List_key_0to1000.pkl', 'rb') as f:
    List_key = pickle.load(f)

In [177]:
len(List_key)

1000

In [178]:
txt_key = []
for i in range(1000): 
 f = open('/content/drive/MyDrive/DADS7202_final/Chart2text/captions/'+str(i)+'.txt', 'r')
 content = f.read()
 content = re.sub('\s+',' ',content)
 txt_key.append(content)
len(txt_key)

1000

In [179]:
import pickle

with open('txt_key_0to1000.pkl', 'wb') as f:
    pickle.dump(txt_key, f)

Load

In [180]:
import pickle

with open('txt_key_0to1000.pkl', 'rb') as f:
    txt_key = pickle.load(f)

In [181]:
print(List_key[0])

Quarter Number of users in millions Q4 '19 2498 Q3 '19 2449 Q2 '19 2414 Q1 '19 2375 Q4 '18 2320 Q3 '18 2271 Q2 '18 2234 Q1 '18 2196 Q4 '17 2129 Q3 '17 2072 Q2 '17 2006 Q1 '17 1936 Q4 '16 1860 Q3 '16 1788 Q2 '16 1712 Q1 '16 1654 Q4 '15 1591 Q3 '15 1545 Q2 '15 1490 Q1 '15 1441 Q4 '14 1393 Q3 '14 1350 Q2 '14 1317 Q1 '14 1276 Q4 '13 1228 Q3 '13 1189 Q2 '13 1155 Q1 '13 1110 Q4 '12 1056 Q3 '12 1007 Q2 '12 955 Q1 '12 901 Q4 '11 845 Q3 '11 800 Q2 '11 739 Q1 '11 680 Q4 '10 608 Q3 '10 550 Q2 '10 482 Q1 '10 431 Q4 '09 360 Q3 '09 305 Q2 '09 242 Q1 '09 197 Q3 '08 100


In [182]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [183]:
# Create a list of lists that represents the data
train_data = [List_key[:600] ,txt_key[:600]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[600:800] ,txt_key[600:800]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[800:] ,txt_key[800:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(dataset)  # Output: Dataset object with train_data
print(type(dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 600
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 200
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 200
})}
<class 'dict'>


In [184]:
from datasets import DatasetDict

dataset_dict = DatasetDict(dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 600
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 200
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 200
    })
})


In [185]:
print(dataset['train'][1])

{'key': "NBA player Number of points scored Kareem Abdul-Jabbar 38387 Karl Malone 36928 LeBron James* 33655 Kobe Bryant 33643 Michael Jordan 32292 Dirk Nowitzki 31560 Wilt Chamberlain 31419 Shaquille O'Neal 28596 Moses Malone 27409 Elvin Hayes 27313 Hakeem Olajuwon 26946 Oscar Robertson 26710 Dominique Wilkins 26668 Tim Duncan 26496 Paul Pierce 26397 John Havlicek 26395 Kevin Garnett 26071 Carmelo Anthony* 26067 Vince Carter* 25623 Alex English 25613 Reggie Miller 25279 Jerry West 25192 Patrick Ewing 24815 Ray Allen 24505 Allen Iverson 24368 Charles Barkley 23757 Robert Parish 23334 Adrian Dantley 23177 Dwyane Wade 23165 Elgin Baylor 23149 Kevin Durant* 22940", 'text': "How many points does LeBron James have ? In January 2020 , LeBron James passed Kobe Bryant on the NBA all-time scoring list , scoring 29 points to in a regular season game against the Philadelphia 76ers . If James continues to produce at about his career average , he has a shot to move up even further and pass Karl Malo

In [186]:
example = dataset_dict['train'][1]

print("Key:", example["key"])
print("Text:", example["text"])

Key: NBA player Number of points scored Kareem Abdul-Jabbar 38387 Karl Malone 36928 LeBron James* 33655 Kobe Bryant 33643 Michael Jordan 32292 Dirk Nowitzki 31560 Wilt Chamberlain 31419 Shaquille O'Neal 28596 Moses Malone 27409 Elvin Hayes 27313 Hakeem Olajuwon 26946 Oscar Robertson 26710 Dominique Wilkins 26668 Tim Duncan 26496 Paul Pierce 26397 John Havlicek 26395 Kevin Garnett 26071 Carmelo Anthony* 26067 Vince Carter* 25623 Alex English 25613 Reggie Miller 25279 Jerry West 25192 Patrick Ewing 24815 Ray Allen 24505 Allen Iverson 24368 Charles Barkley 23757 Robert Parish 23334 Adrian Dantley 23177 Dwyane Wade 23165 Elgin Baylor 23149 Kevin Durant* 22940
Text: How many points does LeBron James have ? In January 2020 , LeBron James passed Kobe Bryant on the NBA all-time scoring list , scoring 29 points to in a regular season game against the Philadelphia 76ers . If James continues to produce at about his career average , he has a shot to move up even further and pass Karl Malone with a

In [24]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [187]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

*************** เปลี่ยน tokenizer เป็นตาม huggingface เลือก model ****************

In [188]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")  ## edit here

prefix = "Text generation from keyword: "
max_input_length = 512
max_target_length = 256

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

In [189]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [190]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [191]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [30]:
!pip install -q pytorch-lightning wandb

******* ปรับ epoch กับ model เป็นตาม huggingface **********

### Create CodeT5 (model1)

In [192]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=200, warmup_steps=100):  ## edit here
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")  ## edit here
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [193]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [194]:
tokenizer.decode(batch['input_ids'][0])

'Text generation from keyword: Month Net worth in billion U.S. dollars Jeff Bezos, Amazon 131.0 Mark Zuckerberg, Facebook 62.3 Larry Page, Google 50.8 Sergey Brin, Google 49.8 Ma Huateng, Tencent 38.8 Jack Ma, Alibaba 37.3 William Ding, Netease 14.7 Colin Huang, Pinduoduo 13.5 Zhang Zhidong, Tencent 13.3 Eric Schmidt, Google 12.9 Pierre Omidyar, eBay 11.4 Dustin Moskovitz, Facebook 11.1 Eduardo Saverin, Facebook 9.7 Jan Koum, WhatsApp 9.6 Robin Li, Baidu 9.6 Joseph Tsai, Alibaba 9.5 Kim Jung-Ju, Nexon 6.5 Richard Liu, JD.com 6.0 Andreas von Bechtolsheim, Google 6.4 David Cheriton, Google 5.8</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [195]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'The ranking of the richest billionaires who earned their fortune in the internet business is headed by Amazon founder Jeff Bezos, who is sitting in first place with a personal net worth of 131 billion U.S. dollars. Facebook founder Mark Zuckerberg was ranked second with 62.3 billion U.S. dollars. Both Bezos and Zuckerberg are also featured in the top 5 ranking of the richest Americans of 2019. Internet billionaires Millions of entrepreneurs are hoping that their company will be the next big thing, and for many startups going public or being acquired by an industry giant are a safe bet to generate riches. The ranking of the richest internet billionaires is dominated by founders and early investors of global online companies including Amazon, Google and Facebook, or startups that were sold to major companies. A popular example of a successful sales is mobile messaging app WhatsApp, which was sold to Facebook in 2014 for an eyewatering 19 billion U.S. dollars, making founders Jan Koum an

In [196]:
model = CodeT5()

### Train (fit) model1

***** ถ้าใช้ k2t ให้ #logger=wandb_logger ด้วย **********

In [197]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

'''
import sys
import signal

save_directory = "/content/drive/MyDrive/DADS7202_final/model1"

def save_and_exit(signal, frame):
    model.model.save_pretrained(save_directory)
    sys.exit(0)

signal.signal(signal.SIGINT, save_and_exit)

'''

wandb_logger = WandbLogger(name='codet5-finetune-code-summarization-ruby-shuffle', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=10,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  logger=wandb_logger, 
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

#### check layers and weight in model

In [198]:
# Access the state_dict attribute to get a dictionary of weights
weights = model.state_dict()

# Print the keys and shapes of the weights
#for key, value in weights.items():
    #print(key, value.shape)

In [114]:
# Get the weights of the first layer
layer_weights = weights['model.decoder.final_layer_norm.weight']

# Print the shape of the weights
print(layer_weights)

tensor([ 1.7973e-01,  3.3264e-01,  1.8100e-01,  1.4566e-01,  4.2287e-03,
         1.3380e-01,  2.7082e-01,  2.6396e-01,  6.2398e-01,  1.3055e-01,
         1.4374e-01,  3.2198e-01,  1.1969e-01,  1.2348e-01,  1.9008e+00,
         1.4036e-01,  1.2550e-01,  1.9818e-01,  1.5188e-01,  1.5158e-01,
         1.4145e-01,  9.8537e-02,  1.9587e-01,  8.4210e-01,  1.1235e-01,
         1.9252e-01,  1.2798e-01,  5.8284e-01,  1.1554e-01,  2.3608e-01,
         1.4415e-01,  2.3927e-01,  1.8415e-01,  1.3343e-01,  4.1885e-01,
         2.3569e-01,  1.4731e-01,  1.4717e-01,  1.4244e-01,  1.7808e-01,
         1.2920e-01,  1.1700e-01,  3.3559e-01,  1.7968e-01,  1.7388e-01,
         1.4171e-01,  3.2423e-01,  1.7452e-01,  1.5451e-01,  1.2819e-01,
         1.5986e-01,  1.7688e-01,  2.2544e-01,  4.3995e-01,  3.0700e-01,
         1.5122e-01,  1.7155e-01,  1.9026e-01,  1.2015e-01,  2.7567e-01,
         1.1972e-01,  2.8066e-01,  2.1101e-01,  1.3057e-01,  1.2070e-01,
         1.9079e-01,  1.5949e-01,  1.1000e-01,  9.1

************* ตั้ง path ที่จะ save model อันแรก ***********

In [199]:
save_directory = "/content/drive/MyDrive/DADS7202_final/model1-1000d-200e-256-v1"
 # save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [200]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 200
})


In [201]:
test_example = dataset2['test'][0]
print("Key:", test_example['key'])

Key: Year Share of respondents 2019 73% 2018 69% 2017 63% 2016 60% 2015 55% 2014 53% 2013 50% 2012 47% 2011 44% 2010 42% 2009 41% 2008 35% 2007 30%


In [202]:
test_example = dataset2['test'][0]
print("Text:", test_example['text'])

Text: This statistic illustrates the share of individuals using online banking regularly in Great Britain from 2007 to 2018 . Banking and financial services are typically among the first industries to embrace technological advances ; starting as far back in time as the appearance of automated teller machines ( ATMs ) and credit cards . Ever since the first secure Internet systems for financial operations became available on a large scale , accessing of one 's bank account online to carry out transactions , pay off credit cards , establish credit lines or invest savings became one of the fastest growing areas of Internet activities . Mobile banking has become the new norm In the UK , online banking use increased during the period under observation : already 30 percent of respondents reported accessing their bank account online within three months prior to the survey in 2007 . The growth in this activity area was steady over time , and the share rose to 69 percent of individuals who used

In [ ]:
#from transformers import T5ForConditionalGeneration
#model = T5ForConditionalGeneration.from_pretrained(save_directory)

**************** AutoModelForSeq2SeqLM ของ แต่ละ model ให้ดูตาม huggingface ******************

In [203]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

### Generate data from own model

In [204]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=150, max_length=200)
print("Generated :", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated : The statistic shows the share of respondents in the United States of America by percentage level in 2019. In 2019, the share of respondents in the United States was 73%. In comparison, the majority of respondents in the United States were in the United States. In 2018, 63 percent of respondents were in the U.S.. The majority of respondents in the United States were in the U.S.. The majority of respondents in the United States were in the U.S.. The majority of respondents in the U.S. were male, who were 65 years old, and 69 percent of respondents were male in 2017. In 2018, the majority of respondents in the United States were male. The majority of respondents in the United States were male. The majority of respondents in the United States were male. The majority of respondents in the United States were male. The majority of respondents in the


Create list of predict data and ref data

In [207]:
predict = []
refer = []
for i in range(200):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=150, max_length=200)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

In [208]:
print(len(predict))
print(len(refer))

200
200


### Evaluation

In [124]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [209]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.07053803536021955, 'precisions': [0.3869311193111931, 0.13159034653465346, 0.061083437110834374, 0.03568295739348371], 'brevity_penalty': 0.6872470113773886, 'length_ratio': 0.727240199476709, 'translation_length': 32520, 'reference_length': 44717}


In [126]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [210]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(predict, refer, avg = True)
print(scores)

{'rouge-1': {'r': 0.18917286590159701, 'p': 0.4849704761548262, 'f': 0.2508952596153873}, 'rouge-2': {'r': 0.07190589612728923, 'p': 0.16910521701829193, 'f': 0.08828661734331121}, 'rouge-l': {'r': 0.1805462926197091, 'p': 0.46069526793896287, 'f': 0.23887516967799136}}


## Create data from Ielt DATASET

*********** แก้ path ************

In [ ]:
List_key = []
for i in range(len(os.listdir('/content/drive/MyDrive/DADS7202_final/IELTS/table'))): 
  src = '/content/drive/MyDrive/DADS7202_final/IELTS/table/'+str(i)+'.png'
  img = cv2.imread(src)
  # Adding custom options
  custom_config = r'--psm 1'
  text = pytesseract.image_to_string(img, config=custom_config)
  #print(text)
  text_split = text.split('\n')
  #print(text_split)
  while '' in text_split:
    text_split.remove('')
  while ' ' in text_split:
    text_split.remove(' ')
  key = ' '.join(text_split[:-2])
  List_key.append(key)
len(List_key)

In [54]:
import pickle

with open('List_key_ielts.pkl', 'wb') as f:
    pickle.dump(List_key, f)

In [211]:
import pickle

with open('List_key_ielts.pkl', 'rb') as f:
    List_key = pickle.load(f)

In [130]:
txt_key = []
for i in range(len(os.listdir('/content/drive/MyDrive/DADS7202_final/IELTS/captions'))): 
  f = open('/content/drive/MyDrive/DADS7202_final/IELTS/captions/'+str(i)+'.txt', 'r')
  content = f.read()
  content = re.sub('\s+',' ',content)
  txt_key.append(content)
len(txt_key)

201

In [58]:
import pickle

with open('txt_key_ielts.pkl', 'wb') as f:
    pickle.dump(txt_key, f)

In [212]:
import pickle

with open('txt_key_ielts.pkl', 'rb') as f:
    txt_key = pickle.load(f)

In [213]:
import pyarrow as pa

# Create a list of lists that represents the data
train_data = [List_key[:121] ,txt_key[:121]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[121:161] ,txt_key[121:161]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[161:] ,txt_key[161:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
ielts_dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(ielts_dataset)  # Output: Dataset object with train_data
print(type(ielts_dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 121
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 40
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 40
})}
<class 'dict'>


In [214]:
print(ielts_dataset['train'][0])

{'key': 'Holiday leisure activities Number of hours |At the beach Reading books Chinese tourists 3 8 American tourists 4 5. Turkish tourists 5 4', 'text': 'The bar chart compares the amount of time each day that people from four different countries spent doing activities while on holiday in Greece in August 2019. The activities were going to the beach, visiting different attractions, and reading. Overall, the tourists spent the most time reading books while they spent the least time visiting places. The Chinese tourists spent the longest reading books with a total of eight hours per day. By contrast, the Brazilians spent the longest at the beach with a total of six hours. Both the American and the Turkish tourists spent the same number of hours every day visiting places with a total of three hours each. Similarly, the Turkish and Brazilian groups spent a total of four hours each reading books. The Chinese tourists spent twice the amount of time reading books as the Turkish and Brazilia

In [215]:
from datasets import DatasetDict

dataset_dict = DatasetDict(ielts_dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 121
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 40
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 40
    })
})


**Train ielts data**

In [64]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

*********** แก้ tokenizer ตาม model เรา **********

In [216]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

prefix = "Text generation from keyword: "
max_input_length = 512
max_target_length = 256

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

In [217]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [218]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [219]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [69]:
!pip install -q pytorch-lightning wandb

### Create CodeT5 (model2)

*********** แก้ AutoModelForSeq2SeqLM ตามแต่ละโมเดลตาม huggingface ********

********* แก้ epoch ***********

In [220]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=200, warmup_steps=100):  #edit here
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)  #edit here
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [ ]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [221]:
tokenizer.decode(batch['input_ids'][0])

'Text generation from keyword: Global sales of games software, CDs and DVD or video billion 2000 2001 2002 2003 Games software 13 15 7 20 DVD/video 18 21 26 30</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [222]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'The chart shows the changes in the sales of video material / DVDs, games software and CDs around the world in billions of dollars over a three-year period. It can be seen that the sales of videos / DVDs and games software have increased, while the sales of CDs have gone down slightly. Between 2000 and 2003, the sale of videos and DVDs rose by approximately 13 billion dollars. In 2000, just under 20 billion dollars worth of these items were sold, but in 2003, this figure had risen to a little over 30 billion dollars. The sales of games software also rose during this period, but less sharply. Sales increased from about 13 billion dollars in 2000 to just under 20 billion dollars three years later. By contrast, during the same time period, the sale of CDs fell from 35 billion dollars in 2000 to about 32.5 billion dollars in 2003.</s>'

Create new model to train again

In [223]:
model = CodeT5()

### Train (fit) model2

In [224]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

wandb_logger = WandbLogger(name='codet5-finetune-code-summarization-ruby-shuffle', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=10,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  #logger=wandb_logger, 
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

******* set new directory ที่จะsaveใน model อันใหม่หลัง train กับ ielts แล้ว ********

In [225]:
save_directory = "/content/drive/MyDrive/DADS7202_final/model2-1000d-200e-256-v1"
 # save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [226]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 40
})


In [227]:
test_example = dataset2['test'][0]
print("Key:", test_example['key'])

Key: % of population 65 andover 1980 2000 2030 Cannada 10% 15% 22% Germany 15% 20% 25%


In [228]:
test_example = dataset2['test'][0]
print("Text:", test_example['text'])

Text: The graph details the percentages of elderly people in Canada, Germany, and the UK in the past with projections for the future. Looking from an overall perspective, it is readily apparent that Elderly populations have risen and are predicted to continue to grow steadily in all countries. Germany has and will continue to have the highest proportion, while Canada will likely overtake the UK. In 1980, Germany had the greatest percentage of elderly at 15%, slightly above the UK and 5% higher than Canada. By 2000, percentages for Germany and Canada both surged by 5% (to 20% and 15%, respectively), compared to the UK which rose marginally to equal with Canada. By 2030, Germany is expected to rise by another 5% to 25%, with Canada surpassing the United Kingdom and finishing at 22% and the latter’s figures lagging behind slightly at just under 20%.


******** แก้ AutoModelForSeq2SeqLM ตาม huggingface

### Generate data from ielts model

In [229]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

In [230]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=150, max_length=200)
print("Generated :", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated : The Canadian population grew by a little over a decade from 1980 to 2030. In 1980, the proportion of the population of Cannada was around 10% of the population, with a decline of 10% from 20% in 1980 to 20% in 2030. The decline in the proportion of the population of Canada was expected to be expected to continue to grow by 2050. The decline in the proportion of the population of Canada was expected to continue to grow by 2050. The decline in the proportion of the population of Canada was expected to continue to grow by 2050. The decline in the number of people aged 65 and over was expected to grow by 2050. In 1980, Germany had a slight increase from 15% to 20%. In contrast, Germany had a slight increase from 15% to 20% in 2000. In 2000, Germany had a slight increase in the proportion of the population of Canada in 1980 to 20%.


### Evaluation

In [231]:
predict = []
refer = []
for i in range(40):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=150, max_length=200)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

In [232]:
print(len(predict))
print(len(refer))

40
40


In [84]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [233]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.041301925436152104, 'precisions': [0.3489219114219114, 0.07810668229777257, 0.02048938679245283, 0.00741399762752076], 'brevity_penalty': 0.9156317400646634, 'length_ratio': 0.9189985272459499, 'translation_length': 6864, 'reference_length': 7469}


In [ ]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [234]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(predict, refer, avg = True)
print(scores)

{'rouge-1': {'r': 0.1830691986950421, 'p': 0.47196248303070076, 'f': 0.2594124479678155}, 'rouge-2': {'r': 0.046020798338071975, 'p': 0.10898166915269958, 'f': 0.06337584848648894}, 'rouge-l': {'r': 0.17077858819417072, 'p': 0.4394792410902807, 'f': 0.2419545862604628}}
